In [1]:
import pandas as pd

In [2]:
# Load the rawdata
raw_df = pd.read_excel("rawdata.xlsx")

In [3]:
raw_df.head()

,date,time,sensor,location,number,activity,position,location.1
0,2024-01-16,10:10:30,0.0,A1,1,placed,inside,A1
1,2024-01-16,10:12:30,1.0,A2,1,picked,inside,A2
2,2024-01-16,10:28:30,1.0,A3,1,picked,Inside,A3
3,2024-01-16,10:38:20,0.0,A4,1,placed,Inside,A4
4,2024-01-16,10:45:20,1.0,A5,1,placed,inside,A5


In [4]:
# To convert the data column to date format
raw_df['date'] = pd.to_datetime(raw_df['date']) 

##### Calculate the datewise duration for inside and outside positions

In [11]:
import pandas as pd

def generate_output(df):

    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])

    
    # To Calculate the datewise total duration for each inside and outside position
    duration_inside = df[df['position'] == 'inside'].groupby([pd.Grouper(key='date'), 'time'])['datetime'].apply(lambda x: x.max() - x.min()).groupby(level=0).sum()
    duration_outside = df[df['position'] != 'inside'].groupby([pd.Grouper(key='date'), 'time'])['datetime'].apply(lambda x: x.max() - x.min() if not x.empty else pd.Timedelta(0)).groupby(level=0).sum()

    # Calculate the datewise number of picking and placing activities
    picking_count = df[df['activity'] == 'picked'].groupby('date').size()
    placing_count = df[df['activity'] == 'placed'].groupby('date').size()

    # Combine the results into a single dataframe
    output = pd.DataFrame({'date': duration_inside.index,
                           'duration_inside': duration_inside.dt.total_seconds() / 3600,
                           'duration_outside': duration_outside.dt.total_seconds() / 3600,
                           'picking_count': picking_count,
                           'placing_count': placing_count})

    # Drop the 'datetime' column from the output
    output = output.drop('datetime', axis=1, errors='ignore')

    return output